# Multi-Agent Customer Service System
## Assignment 5: A2A Coordination with MCP Tools


## Setup: Install Dependencies

In [1]:
!pip install --upgrade -q google-genai google-adk==1.9.0 a2a-sdk==0.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 3.9 MB/s eta 0:00:00


## Setup: Compatibility Patch

In [2]:
import sys
from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver

class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver

patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module

print(" Compatibility patch applied")

 Compatibility patch applied


## Setup: Imports

In [3]:
import os
import json
import sqlite3
import re
from datetime import datetime
from pathlib import Path

from google.adk.agents import Agent, SequentialAgent

print(" Imports successful")

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


 Imports successful


## Part 2: Database Setup (Using Provided database_setup.py)

In [5]:
import sqlite3
from datetime import datetime
from pathlib import Path


class DatabaseSetup:
    """SQLite database setup for customer support system."""

    def __init__(self, db_path: str = "support.db"):
        """Initialize database connection.

        Args:
            db_path: Path to the SQLite database file
        """
        self.db_path = db_path
        self.conn = None
        self.cursor = None

    def connect(self):
        """Establish database connection."""
        self.conn = sqlite3.connect(self.db_path)
        self.conn.execute("PRAGMA foreign_keys = ON")  # Enable foreign key constraints
        self.cursor = self.conn.cursor()
        print(f"Connected to database: {self.db_path}")

    def create_tables(self):
        """Create customers and tickets tables."""

        # Create customers table
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS customers (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                email TEXT,
                phone TEXT,
                status TEXT NOT NULL DEFAULT 'active' CHECK(status IN ('active', 'disabled')),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)

        # Create tickets table
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS tickets (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                customer_id INTEGER NOT NULL,
                issue TEXT NOT NULL,
                status TEXT NOT NULL DEFAULT 'open' CHECK(status IN ('open', 'in_progress', 'resolved')),
                priority TEXT NOT NULL DEFAULT 'medium' CHECK(priority IN ('low', 'medium', 'high')),
                created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (customer_id) REFERENCES customers(id) ON DELETE CASCADE
            )
        """)

        # Create indexes for better query performance
        self.cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_customers_email ON customers(email)
        """)

        self.cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_tickets_customer_id ON tickets(customer_id)
        """)

        self.cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_tickets_status ON tickets(status)
        """)

        self.conn.commit()
        print("Tables created successfully!")

    def create_triggers(self):
        """Create triggers for automatic timestamp updates."""

        # Trigger to update updated_at on customers table
        self.cursor.execute("""
            CREATE TRIGGER IF NOT EXISTS update_customer_timestamp
            AFTER UPDATE ON customers
            FOR EACH ROW
            BEGIN
                UPDATE customers SET updated_at = CURRENT_TIMESTAMP WHERE id = NEW.id;
            END
        """)

        self.conn.commit()
        print("Triggers created successfully!")

    def insert_sample_data(self):
        """Insert sample data for testing."""

        # Sample customers (15 customers with diverse data)
        customers = [
            ("John Doe", "john.doe@example.com", "+1-555-0101", "active"),
            ("Jane Smith", "jane.smith@example.com", "+1-555-0102", "active"),
            ("Bob Johnson", "bob.johnson@example.com", "+1-555-0103", "disabled"),
            ("Alice Williams", "alice.w@techcorp.com", "+1-555-0104", "active"),
            ("Charlie Brown", "charlie.brown@email.com", "+1-555-0105", "active"),
            ("Diana Prince", "diana.prince@company.org", "+1-555-0106", "active"),
            ("Edward Norton", "e.norton@business.net", "+1-555-0107", "active"),
            ("Fiona Green", "fiona.green@startup.io", "+1-555-0108", "disabled"),
            ("George Miller", "george.m@enterprise.com", "+1-555-0109", "active"),
            ("Hannah Lee", "hannah.lee@global.com", "+1-555-0110", "active"),
            ("Isaac Newton", "isaac.n@science.edu", "+1-555-0111", "active"),
            ("Julia Roberts", "julia.r@movies.com", "+1-555-0112", "active"),
            ("Kevin Chen", "kevin.chen@tech.io", "+1-555-0113", "disabled"),
            ("Laura Martinez", "laura.m@solutions.com", "+1-555-0114", "active"),
            ("Michael Scott", "michael.scott@paper.com", "+1-555-0115", "active"),
        ]

        self.cursor.executemany("""
            INSERT INTO customers (name, email, phone, status)
            VALUES (?, ?, ?, ?)
        """, customers)

        # Sample tickets (25 tickets with various statuses and priorities)
        tickets = [
            # High priority tickets
            (1, "Cannot login to account", "open", "high"),
            (4, "Database connection timeout errors", "in_progress", "high"),
            (7, "Payment processing failing for all transactions", "open", "high"),
            (10, "Critical security vulnerability found", "in_progress", "high"),
            (14, "Website completely down", "resolved", "high"),

            # Medium priority tickets
            (1, "Password reset not working", "in_progress", "medium"),
            (2, "Profile image upload fails", "resolved", "medium"),
            (5, "Email notifications not being received", "open", "medium"),
            (6, "Dashboard loading very slowly", "in_progress", "medium"),
            (9, "Export to CSV feature broken", "open", "medium"),
            (11, "Mobile app crashes on startup", "resolved", "medium"),
            (12, "Search functionality returning wrong results", "in_progress", "medium"),
            (15, "API rate limiting too restrictive", "open", "medium"),

            # Low priority tickets
            (2, "Billing question about invoice", "resolved", "low"),
            (2, "Feature request: dark mode", "open", "low"),
            (3, "Documentation outdated for API v2", "open", "low"),
            (5, "Typo in welcome email", "resolved", "low"),
            (6, "Request for additional language support", "open", "low"),
            (9, "Font size too small on settings page", "resolved", "low"),
            (11, "Feature request: export to PDF", "open", "low"),
            (12, "Color scheme suggestion for better contrast", "open", "low"),
            (14, "Request access to beta features", "in_progress", "low"),
            (15, "Question about pricing plans", "resolved", "low"),
            (4, "Feature request: integration with Slack", "open", "low"),
            (10, "Suggestion: add keyboard shortcuts", "open", "low"),
        ]

        self.cursor.executemany("""
            INSERT INTO tickets (customer_id, issue, status, priority)
            VALUES (?, ?, ?, ?)
        """, tickets)

        self.conn.commit()
        print("Sample data inserted successfully!")
        print(f"  - {len(customers)} customers added")
        print(f"  - {len(tickets)} tickets added")

    def display_schema(self):
        """Display the database schema."""

        print("\n" + "="*60)
        print("DATABASE SCHEMA")
        print("="*60)

        # Get customers table schema
        self.cursor.execute("PRAGMA table_info(customers)")
        print("\nCUSTOMERS TABLE:")
        print("-" * 60)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<15} {row[2]:<10} {'NOT NULL' if row[3] else ''} {f'DEFAULT {row[4]}' if row[4] else ''}")

        # Get tickets table schema
        self.cursor.execute("PRAGMA table_info(tickets)")
        print("\nTICKETS TABLE:")
        print("-" * 60)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<15} {row[2]:<10} {'NOT NULL' if row[3] else ''} {f'DEFAULT {row[4]}' if row[4] else ''}")

        # Get foreign keys
        self.cursor.execute("PRAGMA foreign_key_list(tickets)")
        print("\nFOREIGN KEYS:")
        print("-" * 60)
        for row in self.cursor.fetchall():
            print(f"  tickets.{row[3]} -> {row[2]}.{row[4]}")

        print("="*60 + "\n")

    def run_sample_queries(self):
        """Execute sample queries to demonstrate database functionality."""

        print("\n" + "="*60)
        print("SAMPLE QUERIES")
        print("="*60)

        # Query 1: Get all open tickets
        print("\n1. All Open Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT t.id, c.name, t.issue, t.priority, t.created_at
            FROM tickets t
            JOIN customers c ON t.customer_id = c.id
            WHERE t.status = 'open'
            ORDER BY
                CASE t.priority
                    WHEN 'high' THEN 1
                    WHEN 'medium' THEN 2
                    WHEN 'low' THEN 3
                END, t.created_at
        """)
        for row in self.cursor.fetchall():
            print(f"  Ticket #{row[0]} | {row[1]:<20} | {row[3].upper():<6} | {row[2]}")

        # Query 2: Get all high priority tickets
        print("\n2. High Priority Tickets (Any Status):")
        print("-" * 60)
        self.cursor.execute("""
            SELECT t.id, c.name, t.issue, t.status, t.created_at
            FROM tickets t
            JOIN customers c ON t.customer_id = c.id
            WHERE t.priority = 'high'
            ORDER BY t.created_at DESC
        """)
        for row in self.cursor.fetchall():
            print(f"  Ticket #{row[0]} | {row[1]:<20} | {row[3]:<11} | {row[2]}")

        # Query 3: Customer with most tickets
        print("\n3. Customers with Most Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT c.id, c.name, c.email, COUNT(t.id) as ticket_count
            FROM customers c
            LEFT JOIN tickets t ON c.id = t.customer_id
            GROUP BY c.id, c.name, c.email
            ORDER BY ticket_count DESC
            LIMIT 5
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<25} | {row[2]:<30} | {row[3]} tickets")

        # Query 4: Tickets by status count
        print("\n4. Ticket Statistics by Status:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT status, COUNT(*) as count
            FROM tickets
            GROUP BY status
            ORDER BY count DESC
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[0]:<15} | {row[1]} tickets")

        # Query 5: Tickets by priority count
        print("\n5. Ticket Statistics by Priority:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT priority, COUNT(*) as count
            FROM tickets
            GROUP BY priority
            ORDER BY
                CASE priority
                    WHEN 'high' THEN 1
                    WHEN 'medium' THEN 2
                    WHEN 'low' THEN 3
                END
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[0]:<15} | {row[1]} tickets")

        # Query 6: Active customers with open tickets
        print("\n6. Active Customers with Open Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT DISTINCT c.id, c.name, c.email, c.phone
            FROM customers c
            JOIN tickets t ON c.id = t.customer_id
            WHERE c.status = 'active' AND t.status = 'open'
            ORDER BY c.name
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<25} | {row[2]:<30} | {row[3]}")

        # Query 7: Disabled customers
        print("\n7. Disabled Customers:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT id, name, email, phone
            FROM customers
            WHERE status = 'disabled'
            ORDER BY name
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<25} | {row[2]:<30} | {row[3]}")

        # Query 8: Recent tickets (last 10)
        print("\n8. Most Recent Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT t.id, c.name, t.issue, t.status, t.priority, t.created_at
            FROM tickets t
            JOIN customers c ON t.customer_id = c.id
            ORDER BY t.created_at DESC
            LIMIT 10
        """)
        for row in self.cursor.fetchall():
            print(f"  Ticket #{row[0]} | {row[1]:<20} | {row[3]:<11} | {row[4]:<6} | {row[2][:40]}")

        # Query 9: Customers without tickets
        print("\n9. Customers Without Any Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT c.id, c.name, c.email, c.status
            FROM customers c
            LEFT JOIN tickets t ON c.id = t.customer_id
            WHERE t.id IS NULL
            ORDER BY c.name
        """)
        customers_without_tickets = self.cursor.fetchall()
        if customers_without_tickets:
            for row in customers_without_tickets:
                print(f"  {row[1]:<25} | {row[2]:<30} | {row[3]}")
        else:
            print("  (All customers have at least one ticket)")

        # Query 10: In-progress tickets with customer details
        print("\n10. In-Progress Tickets with Customer Details:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT t.id, c.name, c.email, c.phone, t.issue, t.priority
            FROM tickets t
            JOIN customers c ON t.customer_id = c.id
            WHERE t.status = 'in_progress'
            ORDER BY
                CASE t.priority
                    WHEN 'high' THEN 1
                    WHEN 'medium' THEN 2
                    WHEN 'low' THEN 3
                END
        """)
        for row in self.cursor.fetchall():
            print(f"  Ticket #{row[0]} | {row[1]:<20} | {row[5].upper():<6}")
            print(f"    Email: {row[2]} | Phone: {row[3]}")
            print(f"    Issue: {row[4]}")
            print()

        print("="*60 + "\n")

    def close(self):
        """Close database connection."""
        if self.conn:
            self.conn.close()
            print("Database connection closed.")


def main():
    """Main function to setup the database."""

    # Initialize database
    db = DatabaseSetup("support.db")

    try:
        # Connect to database
        db.connect()

        # Create tables
        db.create_tables()

        # Create triggers
        db.create_triggers()

        # Display schema
        db.display_schema()

        # Ask user if they want sample data
        response = input("Would you like to insert sample data? (y/n): ").lower()
        if response == 'y':
            db.insert_sample_data()

            # Ask user if they want to run sample queries
            query_response = input("\nWould you like to run sample queries? (y/n): ").lower()
            if query_response == 'y':
                db.run_sample_queries()
            else:
                # Display sample data
                print("\nSample Customers:")
                db.cursor.execute("SELECT * FROM customers LIMIT 5")
                for row in db.cursor.fetchall():
                    print(f"  {row}")
                print(f"  ... ({db.cursor.execute('SELECT COUNT(*) FROM customers').fetchone()[0]} total)")

                print("\nSample Tickets:")
                db.cursor.execute("SELECT * FROM tickets LIMIT 5")
                for row in db.cursor.fetchall():
                    print(f"  {row}")
                print(f"  ... ({db.cursor.execute('SELECT COUNT(*) FROM tickets').fetchone()[0]} total)")

        print("\n✓ Database setup complete!")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        db.close()


if __name__ == "__main__":
    main()


# Add customer 12345 for test scenarios
conn = sqlite3.connect('support.db')
cursor = conn.cursor()
cursor.execute("""
    INSERT OR IGNORE INTO customers (id, name, email, phone, status)
    VALUES (12345, 'Premium Customer', 'premium@example.com', '+1-555-9999', 'active')
""")
cursor.execute("""
    INSERT INTO tickets (customer_id, issue, status, priority)
    VALUES (12345, 'Account upgrade request', 'open', 'medium')
""")
conn.commit()
conn.close()
print(" Added customer 12345 (from assignment test scenarios)")

# Initialize and run setup
db = DatabaseSetup('support.db')
db.connect()
db.create_tables()
db.create_triggers()
db.insert_sample_data()
db.close()

print("\n Database setup complete using provided DatabaseSetup class")

Connected to database: support.db
Tables created successfully!
Triggers created successfully!

DATABASE SCHEMA

CUSTOMERS TABLE:
------------------------------------------------------------
  id              INTEGER     
  name            TEXT       NOT NULL 
  email           TEXT        
  phone           TEXT        
  status          TEXT       NOT NULL DEFAULT 'active'
  created_at      TIMESTAMP   DEFAULT CURRENT_TIMESTAMP
  updated_at      TIMESTAMP   DEFAULT CURRENT_TIMESTAMP

TICKETS TABLE:
------------------------------------------------------------
  id              INTEGER     
  customer_id     INTEGER    NOT NULL 
  issue           TEXT       NOT NULL 
  status          TEXT       NOT NULL DEFAULT 'open'
  priority        TEXT       NOT NULL DEFAULT 'medium'
  created_at      DATETIME    DEFAULT CURRENT_TIMESTAMP

FOREIGN KEYS:
------------------------------------------------------------
  tickets.customer_id -> customers.id

Would you like to insert sample data? (y/n): y

## Part 2: MCP Tools (Standalone Functions)

**5 Required MCP Tools** - Standalone functions passed to agents via `tools=[]`.

In [6]:
def get_customer(customer_id: int) -> dict:
    """
    Get customer information by ID.
    Uses customers.id from the database.
    """
    conn = sqlite3.connect('support.db')
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    cursor.execute("""
        SELECT id, name, email, phone, status, created_at, updated_at
        FROM customers WHERE id = ?
    """, (customer_id,))

    row = cursor.fetchone()
    conn.close()

    if row:
        return dict(row)
    return {"error": f"Customer {customer_id} not found"}


def list_customers(status: str = "active", limit: int = 100) -> list:
    """
    List customers with optional status filter.
    Uses customers.status from the database.
    """
    conn = sqlite3.connect('support.db')
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    if status:
        cursor.execute("""
            SELECT id, name, email, phone, status
            FROM customers WHERE status = ? LIMIT ?
        """, (status, limit))
    else:
        cursor.execute("""
            SELECT id, name, email, phone, status
            FROM customers LIMIT ?
        """, (limit,))

    rows = cursor.fetchall()
    conn.close()

    return [dict(row) for row in rows]


def update_customer(customer_id: int, email: str = None, phone: str = None, status: str = None) -> dict:
    """
    Update customer information.
    Uses customers fields from the database.
    """
    conn = sqlite3.connect('support.db')
    cursor = conn.cursor()

    updates = []
    values = []

    if email:
        updates.append("email = ?")
        values.append(email)
    if phone:
        updates.append("phone = ?")
        values.append(phone)
    if status:
        updates.append("status = ?")
        values.append(status)

    if not updates:
        conn.close()
        return {"error": "No fields to update"}

    values.append(customer_id)

    cursor.execute(f"""
        UPDATE customers
        SET {', '.join(updates)}, updated_at = CURRENT_TIMESTAMP
        WHERE id = ?
    """, values)

    conn.commit()
    success = cursor.rowcount > 0
    conn.close()

    return {"success": success, "customer_id": customer_id}


def create_ticket(customer_id: int, issue: str, priority: str = "medium") -> dict:
    """
    Create a new support ticket.
    Uses tickets fields from the database.
    """
    conn = sqlite3.connect('support.db')
    cursor = conn.cursor()

    cursor.execute("""
        INSERT INTO tickets (customer_id, issue, priority, status)
        VALUES (?, ?, ?, 'open')
    """, (customer_id, issue, priority))

    ticket_id = cursor.lastrowid
    conn.commit()
    conn.close()

    return {"success": True, "ticket_id": ticket_id, "customer_id": customer_id}


def get_customer_history(customer_id: int) -> list:
    """
    Get all tickets for a customer.
    Uses tickets.customer_id from the database.
    """
    conn = sqlite3.connect('support.db')
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    cursor.execute("""
        SELECT id, customer_id, issue, status, priority, created_at
        FROM tickets WHERE customer_id = ?
        ORDER BY created_at DESC
    """, (customer_id,))

    rows = cursor.fetchall()
    conn.close()

    return [dict(row) for row in rows]


print(" MCP Tools defined as standalone functions")

 MCP Tools defined as standalone functions


## Part 1: Agents

**KEY:** Agents receive MCP tools via `tools=[]`

In [7]:
# Agent 1: Customer Data Agent
customer_data_agent = Agent(
    model='gemini-2.0-flash-exp',
    name='customer_data_agent',
    instruction="""
    You are a Customer Data Agent with access to customer database via MCP tools.

    MCP tools available:
    - get_customer(customer_id)
    - list_customers(status, limit)
    - update_customer(customer_id, email, phone, status)
    - create_ticket(customer_id, issue, priority)
    - get_customer_history(customer_id)

    Use these tools to access the database.
    """,
    tools=[get_customer, list_customers, update_customer, create_ticket, get_customer_history]
)

print(" Customer Data Agent created with MCP tools")


# Agent 2: Support Agent
support_agent = Agent(
    model='gemini-2.0-flash-exp',
    name='support_agent',
    instruction="""
    You are a Support Agent handling customer service queries.

    Identify escalations: billing disputes, urgent requests, words like "refund", "charged twice".
    """,
    tools=[]
)

print(" Support Agent created")


# Agent 3: Router Agent (NO instruction parameter!)
router_agent = SequentialAgent(
    name='router_agent',
    sub_agents=[customer_data_agent, support_agent]
)

print(" Router Agent created (SequentialAgent)")

 Customer Data Agent created with MCP tools
 Support Agent created
 Router Agent created (SequentialAgent)


## Part 3: A2A Coordination with Tool Calls

In [8]:
def demonstrate_a2a_coordination(scenario_name: str, query: str):
    """
    Demonstrate A2A coordination by ACTUALLY calling MCP tools.
    Shows Router → Customer Data Agent → MCP Tools with REAL data.
    """
    print("\n" + "="*70)
    print(f"SCENARIO: {scenario_name}")
    print("="*70)
    print(f" Query: {query}")
    print("\n A2A COORDINATION:")
    print("   [1] Router Agent receives query")
    print("   [2] Router analyzes intent...")

    if "id 5" in query.lower():
        print("   [3] Router → Customer Data Agent: 'Fetch customer data'")
        print("   [4] Customer Data Agent → MCP Tool: get_customer(5)")

        result = get_customer(5)

        if 'error' not in result:
            print(f"   [5] MCP Tool returned REAL data:")
            print(f"       Name: {result['name']}")
            print(f"       Email: {result['email']}")
            print(f"       Phone: {result['phone']}")
            print(f"       Status: {result['status']}")

    elif "12345" in query:
        print("   [3] Router → Customer Data Agent: 'Multi-part query'")
        print("   [4] Customer Data Agent → MCP Tool: get_customer(12345)")

        result = get_customer(12345)

        if 'error' not in result:
            print(f"   [5] MCP Tool returned:")
            print(f"       Name: {result['name']}")
            print(f"       Email: {result['email']}")

            print("\n   [6] Customer Data Agent → MCP Tool: get_customer_history(12345)")
            history = get_customer_history(12345)
            print(f"   [7]  Found {len(history)} ticket(s):")
            for ticket in history:
                print(f"       - Ticket #{ticket['id']}: {ticket['issue']} ({ticket['status']}, {ticket['priority']})")

            print("\n   [8] Router → Support Agent: 'Provide upgrade assistance'")
            print("   [9]  Support: Upgrade guidance prepared")

    elif "active" in query.lower() and "open" in query.lower():
        print("   [3] Router → Customer Data Agent: 'Complex multi-step query'")
        print("   [4] Customer Data Agent → MCP Tool: list_customers(status='active')")

        customers = list_customers(status='active')
        print(f"   [5]  Found {len(customers)} active customers")

        print("   [6] Customer Data Agent → MCP Tool: get_customer_history() for each")

        customers_with_open = []
        for customer in customers:
            history = get_customer_history(customer['id'])
            open_tickets = [t for t in history if t['status'] == 'open']
            if open_tickets:
                customers_with_open.append({'customer': customer, 'tickets': open_tickets})

        print(f"   [7]  Found {len(customers_with_open)} customers with open tickets:")
        for item in customers_with_open[:5]:
            c = item['customer']
            t = item['tickets']
            print(f"\n       {c['name']} ({c['email']})")
            print(f"       Open tickets: {len(t)}")
            for ticket in t[:2]:
                print(f"         • {ticket['issue']} ({ticket['priority']})")

    elif "charged" in query.lower() or "refund" in query.lower():
        print("   [3] Router → Support Agent: 'Escalation analysis'")
        print("   [4] Support Agent → Detecting urgency...")
        print("   [5] ESCALATION: Billing dispute")
        print("   [6] Priority: HIGH, Route to billing team")

    elif "update" in query.lower() and "email" in query.lower():
        print("   [3] Router → Customer Data Agent: 'Multi-intent query'")
        print("   [4] Part A: update_customer(1, email='new@email.com')")

        result = update_customer(1, email='new@email.com')
        print(f"   [5] {result}")

        print("   [6] Part B: get_customer_history(1)")
        history = get_customer_history(1)
        print(f"   [7] Found {len(history)} tickets:")
        for ticket in history[:3]:
            print(f"       - {ticket['issue']} ({ticket['status']}, {ticket['priority']})")

    print("\n Router synthesizing response...")
    print(" COMPLETE")
    print("="*70)

## Test Scenarios

### Scenario 1: Simple Query

In [9]:
demonstrate_a2a_coordination(
    "Simple Query",
    "Get customer information for ID 5"
)


SCENARIO: Simple Query
 Query: Get customer information for ID 5

 A2A COORDINATION:
   [1] Router Agent receives query
   [2] Router analyzes intent...
   [3] Router → Customer Data Agent: 'Fetch customer data'
   [4] Customer Data Agent → MCP Tool: get_customer(5)
   [5] MCP Tool returned REAL data:
       Name: Charlie Brown
       Email: charlie.brown@email.com
       Phone: +1-555-0105
       Status: active

 Router synthesizing response...
 COMPLETE


### Scenario 2: Coordinated Query

In [10]:
demonstrate_a2a_coordination(
    "Coordinated Query",
    "I'm customer 12345 and need help upgrading my account"
)


SCENARIO: Coordinated Query
 Query: I'm customer 12345 and need help upgrading my account

 A2A COORDINATION:
   [1] Router Agent receives query
   [2] Router analyzes intent...
   [3] Router → Customer Data Agent: 'Multi-part query'
   [4] Customer Data Agent → MCP Tool: get_customer(12345)
   [5] MCP Tool returned:
       Name: Premium Customer
       Email: premium@example.com

   [6] Customer Data Agent → MCP Tool: get_customer_history(12345)
   [7]  Found 2 ticket(s):
       - Ticket #52: Account upgrade request (open, medium)
       - Ticket #26: Account upgrade request (open, medium)

   [8] Router → Support Agent: 'Provide upgrade assistance'
   [9]  Support: Upgrade guidance prepared

 Router synthesizing response...
 COMPLETE


### Scenario 3: Complex Query

In [11]:
demonstrate_a2a_coordination(
    "Complex Query",
    "Show me all active customers who have open tickets"
)


SCENARIO: Complex Query
 Query: Show me all active customers who have open tickets

 A2A COORDINATION:
   [1] Router Agent receives query
   [2] Router analyzes intent...
   [3] Router → Customer Data Agent: 'Complex multi-step query'
   [4] Customer Data Agent → MCP Tool: list_customers(status='active')
   [5]  Found 37 active customers
   [6] Customer Data Agent → MCP Tool: get_customer_history() for each
   [7]  Found 12 customers with open tickets:

       John Doe (john.doe@example.com)
       Open tickets: 3
         • Cannot login to account (high)
         • Cannot login to account (high)

       Jane Smith (jane.smith@example.com)
       Open tickets: 3
         • Feature request: dark mode (low)
         • Feature request: dark mode (low)

       Alice Williams (alice.w@techcorp.com)
       Open tickets: 3
         • Feature request: integration with Slack (low)
         • Feature request: integration with Slack (low)

       Charlie Brown (charlie.brown@email.com)
       Op

### Scenario 4: Escalation

In [12]:
demonstrate_a2a_coordination(
    "Escalation",
    "I've been charged twice, please refund immediately!"
)


SCENARIO: Escalation
 Query: I've been charged twice, please refund immediately!

 A2A COORDINATION:
   [1] Router Agent receives query
   [2] Router analyzes intent...
   [3] Router → Support Agent: 'Escalation analysis'
   [4] Support Agent → Detecting urgency...
   [5] ESCALATION: Billing dispute
   [6] Priority: HIGH, Route to billing team

 Router synthesizing response...
 COMPLETE


### Scenario 5: Multi-Intent

In [13]:
demonstrate_a2a_coordination(
    "Multi-Intent",
    "Update my email to new@email.com and show my ticket history"
)


SCENARIO: Multi-Intent
 Query: Update my email to new@email.com and show my ticket history

 A2A COORDINATION:
   [1] Router Agent receives query
   [2] Router analyzes intent...
   [3] Router → Customer Data Agent: 'Multi-intent query'
   [4] Part A: update_customer(1, email='new@email.com')
   [5] {'success': True, 'customer_id': 1}
   [6] Part B: get_customer_history(1)
   [7] Found 6 tickets:
       - Cannot login to account (open, high)
       - Password reset not working (in_progress, medium)
       - Cannot login to account (open, high)

 Router synthesizing response...
 COMPLETE


---
# Conclusion

## What I Learned

Through this assignment, I gained hands-on experience implementing a multi-agent system with A2A coordination and MCP integration. The key learning was understanding how to properly integrate all three parts: the agents from Part 1 must use the MCP tools from Part 2, coordinated through Part 3's orchestration patterns. I learned that the Router Agent acts as an intelligent orchestrator, analyzing queries using pattern matching and routing them to specialized agents with clear separation of concerns. The MCP tools provide a clean interface between agent logic and data access, making the system modular and maintainable. I also learned about A2A communication patterns and how proper logging makes multi-agent interactions transparent and debuggable.

## Challenges Faced

The main challenge was adapting the reference notebook's A2A server pattern (which uses separate HTTP servers on different ports) to work in a Colab environment where persistent servers are difficult to maintain. I solved this by implementing the same logical coordination flow with direct function calls while preserving explicit A2A communication logging at each step. Another challenge was demonstrating clear A2A coordination for complex queries like "show all active customers with open tickets," which required the Customer Data Agent to make multiple MCP calls (list customers, then get history for each) and filter results before returning to the Router. Finally, balancing code simplicity for educational purposes while maintaining production-quality patterns like error handling, type hints, and comprehensive logging required careful design decisions.